In [1]:
import numpy as np
import pandas as pd
import time
from kafka import KafkaProducer
from sqlalchemy import create_engine


In [2]:
# Explore the original data for Taxi in 2019-1
path = '/Users/mhy/Code/pythonworkspace/Deep-NYC-Bike-Taxi/data/yellow_tripdata_2019-01.csv'
taxi_data = pd.read_csv(path, dtype=str)
taxi_data.head(5)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.50,1,N,151,239,1,7,0.5,0.5,1.65,0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.60,1,N,239,246,1,14,0.5,0.5,1,0,0.3,16.3,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,.00,1,N,236,236,1,4.5,0.5,0.5,0,0,0.3,5.8,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,.00,1,N,193,193,2,3.5,0.5,0.5,0,0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,.00,2,N,193,193,2,52,0,0.5,0,0,0.3,55.55,NaN


In [3]:
taxi_data.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge'],
      dtype='object')

In [4]:
# Discard some unrelated columns to simplify the problem
# Some other attributes may be used later
simplified_taxi_data = taxi_data.drop(['VendorID',  'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'payment_type', 'fare_amount', 'extra', 'total_amount',
                                       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge'], axis=1)

simplified_taxi_data


,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID
0,2019-01-01 00:46:40,2019-01-01 00:53:20,151,239
1,2019-01-01 00:59:47,2019-01-01 01:18:59,239,246
2,2018-12-21 13:48:30,2018-12-21 13:52:40,236,236
3,2018-11-28 15:52:25,2018-11-28 15:55:45,193,193
4,2018-11-28 15:56:57,2018-11-28 15:58:33,193,193
...,...,...,...,...
7667787,2019-01-31 23:57:36,2019-02-01 00:18:39,263,4
7667788,2019-01-31 23:32:03,2019-01-31 23:33:11,193,193
7667789,2019-01-31 23:36:36,2019-01-31 23:36:40,264,264
7667790,2019-01-31 23:14:53,2019-01-31 23:15:20,264,7


In [5]:
# Filter out the data within the manhattan zones
simplified_taxi_data['DOLocationID'] = simplified_taxi_data['DOLocationID'].astype('int64')
simplified_taxi_data['PULocationID'] = simplified_taxi_data['PULocationID'].astype('int64')

manhattan_zones = pd.read_csv("./data-NYCZones/zones/manhattan_zones.csv")
manhattan_zones_id  = list(manhattan_zones["zone_id"])
manhattan_taxi_data = simplified_taxi_data[(simplified_taxi_data['DOLocationID'].isin(manhattan_zones_id)) & (simplified_taxi_data['PULocationID'].isin(manhattan_zones_id))]

# Discard data out of the time range
year = 2019
month = 1
manhattan_taxi_data = manhattan_taxi_data[manhattan_taxi_data["tpep_pickup_datetime"] > f"{year}-%02d-01 00:00:00" % month]
manhattan_taxi_data = manhattan_taxi_data[manhattan_taxi_data["tpep_dropoff_datetime"] > f"{year}-%02d-01 00:00:00" % month]
manhattan_taxi_data = manhattan_taxi_data[manhattan_taxi_data["tpep_pickup_datetime"] < f"{year}-%02d-31 23:59:59" % month]
manhattan_taxi_data = manhattan_taxi_data[manhattan_taxi_data["tpep_dropoff_datetime"] < f"{year}-%02d-31 23:59:59" % month]

# Align time into hour, maybe handled with flink later
manhattan_taxi_data['tpep_pickup_datetime'] = pd.to_datetime(manhattan_taxi_data['tpep_pickup_datetime'])
manhattan_taxi_data['tpep_dropoff_datetime'] = pd.to_datetime(manhattan_taxi_data['tpep_dropoff_datetime'])
manhattan_taxi_data['alignedtime'] = manhattan_taxi_data.tpep_pickup_datetime.dt.floor(f'{60}min')
manhattan_taxi_data = manhattan_taxi_data.sort_values('alignedtime')

In [6]:
manhattan_zones

,zone_id,graph_id,zone_name
0,4,0,Alphabet City
1,12,1,Battery Park
2,13,2,Battery Park City
3,24,3,Bloomingdale
4,41,4,Central Harlem
...,...,...,...
64,246,64,West Chelsea/Hudson Yards
65,249,65,West Village
66,261,66,World Trade Center
67,262,67,Yorkville East


In [ ]:
# Transform the PULocationID to the garph id
# Unused since this function is too slow

# manhattan_taxi_data['PULocationID'] = manhattan_taxi_data['PULocationID'].apply(
#     lambda original_id: manhattan_zones[manhattan_zones['zone_id'] == original_id].graph_id)


In [61]:
# Use join instead to transform the zone id
# Unused here because joining are done in ES

# result = pd.merge(manhattan_taxi_data, manhattan_zones, left_on='PULocationID', right_on='zone_id')
# result['PULocationID'] = result['graph_id']
# result = result.drop(['zone_id', 'graph_id', 'zone_name'], axis=1)

# result = pd.merge(result, manhattan_zones, left_on='DOLocationID', right_on='zone_id')
# result['DOLocationID'] = result['graph_id']
# result = result.drop(['zone_id', 'graph_id', 'zone_name'], axis=1)

In [7]:
# Sort by drop off time to simulate the real-time events
result = manhattan_taxi_data
result = result.sort_values(by=['tpep_dropoff_datetime'])
result

,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID,alignedtime
6489,2019-01-01 00:00:18,2019-01-01 00:01:43,162,162,2019-01-01 00:00:00
10209,2019-01-01 00:00:15,2019-01-01 00:02:07,166,166,2019-01-01 00:00:00
4844,2019-01-01 00:00:51,2019-01-01 00:02:09,90,234,2019-01-01 00:00:00
8556,2019-01-01 00:02:13,2019-01-01 00:02:23,90,90,2019-01-01 00:00:00
11167,2019-01-01 00:02:16,2019-01-01 00:02:30,42,42,2019-01-01 00:00:00
...,...,...,...,...,...
7656512,2019-01-31 23:53:44,2019-01-31 23:59:58,186,68,2019-01-31 23:00:00
7659673,2019-01-31 23:51:06,2019-01-31 23:59:58,90,246,2019-01-31 23:00:00
7660602,2019-01-31 23:54:21,2019-01-31 23:59:58,161,237,2019-01-31 23:00:00
7658376,2019-01-31 23:47:02,2019-01-31 23:59:58,249,230,2019-01-31 23:00:00


In [8]:
taxi_inflow = result.drop(['tpep_pickup_datetime', 'PULocationID', 'alignedtime'], axis=1)
taxi_outflow = result.drop(['tpep_dropoff_datetime', 'DOLocationID', 'alignedtime'], axis=1)

In [9]:
taxi_inflow['tpep_dropoff_datetime'] = taxi_inflow['tpep_dropoff_datetime'].astype(str)
taxi_outflow['tpep_pickup_datetime'] = taxi_outflow['tpep_pickup_datetime'].astype(str)

In [10]:
taxi_inflow

,tpep_dropoff_datetime,DOLocationID
6489,2019-01-01 00:01:43,162
10209,2019-01-01 00:02:07,166
4844,2019-01-01 00:02:09,234
8556,2019-01-01 00:02:23,90
11167,2019-01-01 00:02:30,42
...,...,...
7656512,2019-01-31 23:59:58,68
7659673,2019-01-31 23:59:58,246
7660602,2019-01-31 23:59:58,237
7658376,2019-01-31 23:59:58,230


In [11]:
taxi_outflow

,tpep_pickup_datetime,PULocationID
6489,2019-01-01 00:00:18,162
10209,2019-01-01 00:00:15,166
4844,2019-01-01 00:00:51,90
8556,2019-01-01 00:02:13,90
11167,2019-01-01 00:02:16,42
...,...,...
7656512,2019-01-31 23:53:44,186
7659673,2019-01-31 23:51:06,90
7660602,2019-01-31 23:54:21,161
7658376,2019-01-31 23:47:02,249


In [13]:
# Export data to kafka
producer = KafkaProducer(bootstrap_servers='localhost:9092')

# Write 100 messages to kafka
# The number need to be considered later
N = 1000
for i in range(N):
    # json_string = simplified_taxi_data.iloc[i].to_json()
    inflow_string = taxi_inflow.iloc[i].to_json()
    outflow_string = taxi_outflow.iloc[i].to_json()
    # To send a message
    producer.send('taxi_inflow', inflow_string.encode('utf-8'))
    producer.send('taxi_outflow', outflow_string.encode('utf-8'))
    time.sleep(10)

producer.flush()  # Wait for any outstanding messages to be transmitted and delivery acknowledgments received
producer.close()

KeyboardInterrupt: 

In [ ]:
# Connect to MySQL server
# Replace the following values with your database credentials
USERNAME = 'root'
PASSWORD = 'MySQLroot'
HOST = 'localhost'
DATABASE_NAME = 'traffic'

engine = create_engine(f'mysql+mysqlconnector://{USERNAME}:{PASSWORD}@{HOST}/{DATABASE_NAME}')

# Create a table in MySQL and insert the data
# Convert a dataframe to SQL create table statement
manhattan_taxi_sampled = manhattan_taxi_data[:100]
table_name = 'manhattan_taxi'
manhattan_taxi_sampled.to_sql(table_name, engine, if_exists='replace', index=False)
create_sql = pd.io.sql.get_schema(manhattan_taxi_sampled, table_name, con=engine)
print(create_sql)

In [ ]:
-- The SQL to connect Flink to Kafka in sql_client
CREATE TABLE manhattan_taxi (
	tpep_pickup_datetime TIMESTAMP(3), 
	tpep_dropoff_datetime TIMESTAMP(3), 
	`PULocationID` BIGINT, 
	`DOLocationID` BIGINT, 
    WATERMARK FOR tpep_pickup_datetime AS tpep_pickup_datetime
)WITH (
    'connector' = 'kafka',  -- using kafka connector
    'topic' = 'traffic',  -- kafka topic
    'scan.startup.mode' = 'earliest-offset',  -- reading from the beginning
    'properties.bootstrap.servers' = 'localhost:9092',  -- kafka broker address
    'format' = 'json'  -- the data format is json
);

CREATE TABLE taxi_inflow (
	tpep_dropoff_datetime TIMESTAMP(3), 
	`DOLocationID` BIGINT, 
    WATERMARK FOR tpep_dropoff_datetime AS tpep_dropoff_datetime - INTERVAL '5' SECOND
)WITH (
    'connector' = 'kafka',  -- using kafka connector
    'topic' = 'taxi_inflow',  -- kafka topic
    'scan.startup.mode' = 'earliest-offset',  -- reading from the beginning
    'properties.bootstrap.servers' = 'localhost:9092',  -- kafka broker address
    'format' = 'json'  -- the data format is json
);

CREATE TABLE taxi_outflow (
	tpep_pickup_datetime TIMESTAMP(3), 
	`PULocationID` BIGINT, 
    WATERMARK FOR tpep_pickup_datetime AS tpep_pickup_datetime - INTERVAL '5' SECOND
)WITH (
    'connector' = 'kafka',  -- using kafka connector
    'topic' = 'taxi_outflow',  -- kafka topic
    'scan.startup.mode' = 'earliest-offset',  -- reading from the beginning
    'properties.bootstrap.servers' = 'localhost:9092',  -- kafka broker address
    'format' = 'json'  -- the data format is json
);

In [ ]:
-- Calculate the inflow of each zone
SELECT DATE_FORMAT(tpep_pickup_datetime, '%Y-%m-%d %H:00:00') AS hour, PULocationID, COUNT(*) AS count
FROM manhattan_taxi
GROUP BY hour, PULocationID;

-- Calculate the outflow of each zone
SELECT DATE_FORMAT(tpep_dropoff_datetime, '%Y-%m-%d %H:00:00') AS hour, DOLocationID, COUNT(*) AS count
FROM taxi_inflow
GROUP BY hour, DOLocationID;

In [ ]:
-- Not work
-- FIXME: Bugs for the tumbling window functions
SELECT TUMBLE_START(tpep_dropoff_datetime, INTERVAL '1' HOUR) AS hour_start, 
  DOLocationID, 
  COUNT(*) AS total_dropoffs 
FROM taxi_inflow 
GROUP BY TUMBLE(tpep_dropoff_datetime, INTERVAL '1' HOUR), DOLocationID

-- Work
SELECT DATE_FORMAT(tpep_dropoff_datetime, 'yyyy-mm-dd HH:00:00') AS hour_start, 
    DOLocationID, 
    COUNT(*) AS `count`
FROM taxi_inflow
GROUP BY DATE_FORMAT(tpep_dropoff_datetime, 'yyyy-mm-dd HH:00:00'), DOLocationID;

In [ ]:
# Generate multiple lines of SQL, not used
sql_string = ''
for i in range(68):
    sql_string += f'SUM(CASE WHEN DOLocationID = {i} THEN 1 ELSE 0 END) AS location{i},\n'

print(sql_string[:-2])

In [ ]:
-- Flink SQL to get inflow of columns T, N, Inflow
SELECT 
    DATE_FORMAT(tpep_dropoff_datetime, 'yyyy-MM-dd HH:00:00') AS hour_start, 
    DOLocationID, 
    COUNT(*) as inflow
FROM taxi_inflow
GROUP BY DATE_FORMAT(tpep_dropoff_datetime, 'yyyy-MM-dd HH:00:00'), DOLocationID;

-- Flink SQL to get outflow of columns T, N, outflow
SELECT 
    DATE_FORMAT(tpep_pickup_datetime, 'yyyy-MM-dd HH:00:00') AS hour_start, 
    PULocationID, 
    COUNT(*) as outflow
FROM taxi_outflow
GROUP BY DATE_FORMAT(tpep_pickup_datetime, 'yyyy-MM-dd HH:00:00'), PULocationID;

In [ ]:
-- Elasticsearch
CREATE TABLE taxi_inflow_es (
    hour_start TIMESTAMP, 
	`DOLocationID` BIGINT, 
    inflow BIGINT
) WITH (
    'connector' = 'elasticsearch-7', -- using elasticsearch connector
    'hosts' = 'https://demo0.es.asia-southeast1.gcp.elastic-cloud.com:9243',  -- elasticsearch address
    'username' = 'elastic',
    'password' = 'G1Tqo3onjeIg7G8E6zSIWlq8',
    'index' = 'taxi_inflow_es'  -- elasticsearch index name, similar to database table name
);

INSERT INTO taxi_inflow_es
(SELECT 
    CAST(DATE_FORMAT(tpep_dropoff_datetime, 'yyyy-MM-dd HH:00:00') AS TIMESTAMP) AS hour_start, 
    DOLocationID, 
    COUNT(*) as inflow
FROM taxi_inflow
GROUP BY CAST(DATE_FORMAT(tpep_dropoff_datetime, 'yyyy-MM-dd HH:00:00') AS TIMESTAMP), DOLocationID);

INSERT INTO taxi_inflow_es (SELECT * FROM taxi_inflow);

CREATE TABLE taxi_outflow_es (
    hour_start TIMESTAMP(3), 
	`PULocationID` BIGINT, 
    outflow BIGINT
) WITH (
    'connector' = 'elasticsearch-7', -- using elasticsearch connector
    'hosts' = 'http://localhost:9200',  -- elasticsearch address
    'index' = 'taxi_outflow_es'  -- elasticsearch index name, similar to database table name
);

INSERT INTO taxi_outflow_es
(SELECT 
    DATE_FORMAT(tpep_pickup_time, 'yyyy-MM-dd HH:00:00') AS hour_start, 
    PULocationID, 
    COUNT(*) as outflow
FROM taxi_outflow
GROUP BY DATE_FORMAT(tpep_pickup_datetime, 'yyyy-MM-dd HH:00:00'), PULocationID);

INSERT INTO taxi_outflow_es (SELECT * FROM taxi_outflow);